<a href="https://colab.research.google.com/github/tfranke0814/pytorch-deep-learning/blob/main/01_pytorch_workflow_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch, numpy as np